<a href="https://colab.research.google.com/github/JstnClmnt/ACM-ICPC-Algorithms/blob/master/NLP_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import re
np.random.seed(1)

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import os
 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName+"/"+entry
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles        

In [0]:
!unzip "/content/drive/Team Drives/AI Lords/NLP-Text-Summarization/dataset.zip"

In [0]:
import pandas as pd


dirName = 'data';
    
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)
    

df=pd.DataFrame(columns=["Title","Description","Category"])
# Print the files    
for elem in listOfFiles:
    print("Reading: "+elem)
    file1 = open(elem,"r",encoding="latin-1") 
    sampleNews=file1.read().split("\n")
    newsDesc=""
    for strline in sampleNews[1:len(sampleNews)]:
        newsDesc+=strline
    """
    print("Category: "+elem.split("/")[1])
    print("Title:"+sampleNews[0])
    print("Description:"+newsDesc[0:50])
    """
    dfsample=pd.DataFrame(columns=["Title","Description","Category"],data=[[sampleNews[0],newsDesc,elem.split("/")[1]]])
    df=df.append(dfsample)

In [0]:
df=df.reset_index()
df=df.drop(list(df)[0], axis=1)
df["Description"].head()[0]

In [0]:
df=df[["Title","Description"]]
df.head()

In [0]:
df['Title'] = df['Title'].str.replace("[^a-zA-Z#]"," ")
df['Description'] = df['Description'].str.replace("[^a-zA-Z#]"," ")
df=df.dropna()

In [0]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 20000
tokenizer = Tokenizer(oov_token=1,num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df["Description"])
article_sequences = tokenizer.texts_to_sequences(df["Description"])
art_word_index = tokenizer.word_index
len(art_word_index)


In [0]:
art_word_index_1500 = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_word_index_1500[word] = art_word_index[word]
        counter += 1
print(counter)


In [0]:
tokenizer.fit_on_texts(df["Title"])
summary_sequences = tokenizer.texts_to_sequences(df["Title"])
sum_word_index = tokenizer.word_index
len(sum_word_index)

In [0]:
sum_word_index_1500 = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_word_index_1500[word] = sum_word_index[word]
        counter += 1
print(counter)


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(article_sequences, summary_sequences, test_size=0.20, random_state=42)

print(X_train[0])

In [0]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [0]:
!unzip "glove.6B.zip"

In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
EMB_DIM=300
num_words=VOCAB_SIZE+1
print("Number of Words:"+str(num_words))

#MAX_LENGTH = len(max(X_train, key=len))
#print("Max Length: "+str(MAX_LENGTH))  # 271

MAX_LENGTH = 500
print("Max Length for Sentences: "+str(MAX_LENGTH))
MAX_LENGTH_SUM = len(max(y_train, key=len))
print("Max Length for Summaries: "+str(MAX_LENGTH_SUM))  # 271

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post',truncating="post")
test_sentences_X = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post',truncating="post")
train_sentences_Y= pad_sequences(y_train, maxlen=MAX_LENGTH_SUM, padding='post',truncating="post")
test_sentences_Y= pad_sequences(y_test, maxlen=MAX_LENGTH_SUM, padding='post',truncating="post")
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(train_sentences_Y[0])

In [0]:
embedding_matrix_art=np.zeros((num_words,EMB_DIM))
numNoEmb=0
#print(word2index)
for word,i in art_word_index_1500.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_art[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

In [0]:
embedding_matrix_sum=np.zeros((num_words,EMB_DIM))
#print(word2index)
numNoEmb=0
for word,i in sum_word_index_1500.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_sum[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

In [0]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import concatenate,Input,Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout,Add
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import Constant
import numpy as np
from tensorflow.keras.utils import plot_model

HIDDEN_UNITS=512

"""
Chatbot Inspired Encoder-Decoder-seq2seq
"""
'''
encoder_inputs = Input(shape=(MAX_LENGTH, ))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_art),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_inputs = Input(shape=(MAX_LENGTH, ))
decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_sum),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(num_words, activation='softmax'))(decoder_outputs)
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
'''


"""
Bidirectional LSTM: Others Inspired Encoder-Decoder-seq2seq
"""
encoder_inputs = Input(shape=(MAX_LENGTH,))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_art),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_LSTM_R = LSTM(HIDDEN_UNITS, return_state=True, go_backwards=True)
encoder_outputs_R, state_h_R, state_c_R = encoder_LSTM_R(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

final_h = Add()([state_h, state_h_R])
final_c = Add()([state_c, state_c_R])
encoder_states = [final_h, final_c]

"""
decoder
"""
decoder_inputs = Input(shape=(MAX_LENGTH_SUM,))
decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_sum),input_length=MAX_LENGTH_SUM,trainable=False,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states) 
decoder_dense = TimeDistributed(Dense(num_words, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

model= tf.keras.Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)


model.compile(optimizer=Adam(0.001), loss="categorical_crossentropy", metrics=["accuracy"])

print(model.summary())
plot_model(model, to_file='model_seq2seq.png')

In [0]:
num_samples_train = len(train_sentences_Y)
print(num_samples_train)
decoder_output_data_train = np.zeros(shape=(num_samples_train, MAX_LENGTH_SUM, num_words))

In [0]:
num_samples_test = len(test_sentences_Y)
print(num_samples_test)
decoder_output_data_test = np.zeros(shape=(num_samples_test, MAX_LENGTH_SUM, num_words))

In [0]:
for i, seqs in enumerate(train_sentences_Y):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data_train[i][j][seq] = 1
            
for i, seqs in enumerate(test_sentences_Y):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data_test[i][j][seq] = 1

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

save = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='acc', mode='max')
es_acc = EarlyStopping(monitor='acc', mode='max',verbose=1,patience=10,min_delta=0.003)
es_loss = EarlyStopping(monitor='loss', mode='min',patience=10,verbose=1)
history = model.fit([train_sentences_X, train_sentences_Y], 
                     decoder_output_data_train, 
                     epochs=150, 
                     batch_size=32,
                     validation_data=([test_sentences_X, test_sentences_Y], decoder_output_data_test),
                     callbacks=[save,es_acc,es_loss])

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.yticks(np.arange(0,1,step=0.1))
plt.savefig("acc_lstm.png")

plt.figure( )
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("loss_lstm.png")

In [0]:
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(HIDDEN_UNITS,))
decoder_state_input_c = Input(shape=(HIDDEN_UNITS,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_LSTM(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((MAX_LENGTH_SUM, num_words))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[ -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > MAX_LENGTH_SUM):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((MAX_LENGTH_SUM, num_words))
        target_seq[0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [65]:
print(decode_sequence(train_sentences_X[0:1]))

ValueError: ignored

In [66]:
 it predict the next word again and again.

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
# Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True
# Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
# Update states
        states_value = [h, c]
return decoded_sentence

ValueError: ignored